In [0]:
#common 1st 4 lines

from pyspark import SparkContext

from pyspark.sql import SparkSession

sc=SparkContext.getOrCreate()

spark=SparkSession.builder.appName('CaseStudy Online banking').getOrCreate()


In [0]:
# Load datasets
loans_df = spark.read.csv("/FileStore/tables/loan.csv", header=True, inferSchema=True)
credit_df = spark.read.csv("/FileStore/tables/credit_card.csv", header=True, inferSchema=True)
transactions_df = spark.read.csv("/FileStore/tables/txn.csv", header=True, inferSchema=True)

In [0]:
# Loan Data Analysis
# Number of loans in each category
loan_categories = loans_df.groupBy("Loan Category").count()
print("Loan categories:")
loan_categories.show()

Loan categories:
+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|           HOUSING|   67|
|        TRAVELLING|   53|
|       BOOK STORES|    7|
|       AGRICULTURE|   12|
|         GOLD LOAN|   77|
|  EDUCATIONAL LOAN|   20|
|        AUTOMOBILE|   60|
|          BUSINESS|   24|
|COMPUTER SOFTWARES|   35|
|           DINNING|   14|
|          SHOPPING|   35|
|       RESTAURANTS|   41|
|       ELECTRONICS|   14|
|          BUILDING|    7|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
+------------------+-----+



In [0]:
# Number of people who have taken more than 1 lakh loan
from pyspark.sql.functions import col

high_value_loans = loans_df.filter(col("Loan Amount") > 100000).count()
print(f"Number of people with loans > 1 lakh: {high_value_loans}")

Number of people with loans > 1 lakh: 0


In [0]:
# Number of people with income greater than 60,000
high_income = loans_df.filter(col("Income") > 60000).count()
print(f"Number of people with income > 60,000: {high_income}")


Number of people with income > 60,000: 198


In [0]:
# Number of people with 2+ returned cheques and income less than 50,000
high_risk_customers = loans_df.filter((col(" Returned Cheque") >= 2) & (col("Income") < 50000)).count()
print(f"Number of high-risk customers: {high_risk_customers}")

Number of high-risk customers: 137


In [0]:
# Number of people with 2+ returned cheques and are single
single_high_risk = loans_df.filter((col(" Returned Cheque") >= 2) & (col("Marital Status") == "Single")).count()
print(f"Single high-risk customers: {single_high_risk}")


Single high-risk customers: 0


In [0]:
# Number of people with expenditure over 50,000 a month
high_expenditure = loans_df.filter(col("Expenditure") > 50000).count()
print(f"High expenditure customers: {high_expenditure}")

High expenditure customers: 6


In [0]:
# Credit Data Analysis
# Credit card users in Spain
credit_users_spain = credit_df.filter(col("Geography") == "Spain")
a = credit_users_spain.count()
print(f"Credit card users in Spain: {a}")
credit_users_spain.show()

Credit card users in Spain: 2477
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        5|  15737888| Mitchell|        850|    Spain|Female| 43|     2|125510.82|            1|             1|        79084.1|     0|
|        6|  15574012|      Chu|        645|    Spain|  Male| 44|     8|113755.78|            2|             0|      149756.71|     1|
|       12|  15737173|  Andrews|        497|    Spain|  Male| 24|     3|      0.0|            2|             0|       76390.01|     0|
|       15|  15600882|

In [0]:
print(credit_df.columns)
print(loans_df.columns)



['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'IsActiveMember', 'EstimatedSalary', 'Exited']
['Customer_ID', 'Age', 'Gender', 'Occupation', 'Marital Status', 'Family Size', 'Income', 'Expenditure', 'Use Frequency', 'Loan Category', 'Loan Amount', 'Overdue', ' Debt Record', ' Returned Cheque', ' Dishonour of Bill']


In [0]:
# Transactions Data Analysis
# Maximum withdrawal amount in transactions
from pyspark.sql.functions import max
max_withdrawal = transactions_df.select(max(" WITHDRAWAL AMT ")).collect()[0][0]
print(f"Maximum withdrawal amount: {max_withdrawal}")

Maximum withdrawal amount: 459447546.4


In [0]:
# Minimum withdrawal amount of an account
from pyspark.sql.functions import min
min_withdrawal = transactions_df.select(min(" WITHDRAWAL AMT ")).collect()[0][0]
print(f"Minimum withdrawal amount: {min_withdrawal}")

Minimum withdrawal amount: 0.01


In [0]:
# Maximum deposit amount of an account
max_deposit = transactions_df.select(max(" DEPOSIT AMT ")).collect()[0][0]
print(f"Maximum deposit amount: {max_deposit}")


Maximum deposit amount: 544800000.0


In [0]:
# Minimum deposit amount of an account
min_deposit = transactions_df.select(min(" DEPOSIT AMT ")).collect()[0][0]
print(f"Minimum deposit amount: {min_deposit}")


Minimum deposit amount: 0.01


In [0]:
# Sum of balance in every bank account
from pyspark.sql.functions import sum
total_balance = transactions_df.agg(sum("BALANCE AMT")).collect()[0][0]
print(f"Total account balance: {total_balance}")

Total account balance: -163245212011488.44


In [0]:
print(transactions_df.columns)

['Account No', 'TRANSACTION DETAILS', 'VALUE DATE', ' WITHDRAWAL AMT ', ' DEPOSIT AMT ', 'BALANCE AMT']


In [0]:
# Number of transactions on each date
transactions_by_date = transactions_df.groupBy("VALUE DATE").count()
print("Transactions by date:")
transactions_by_date.show()

Transactions by date:
+----------+-----+
|VALUE DATE|count|
+----------+-----+
| 23-Dec-16|  143|
|  7-Feb-19|   98|
| 21-Jul-15|   80|
|  9-Sep-15|   91|
| 17-Jan-15|   16|
| 18-Nov-17|   53|
| 21-Feb-18|   77|
| 20-Mar-18|   71|
| 19-Apr-18|   71|
| 21-Jun-16|   97|
| 17-Oct-17|  101|
|  3-Jan-18|   70|
|  8-Jun-18|  223|
| 15-Dec-18|   62|
|  8-Aug-16|   97|
| 17-Dec-16|   74|
|  3-Sep-15|   83|
| 21-Jan-16|   76|
|  4-May-18|   92|
|  7-Sep-17|   94|
+----------+-----+
only showing top 20 rows



In [0]:
# List of customers with withdrawal amount more than 1 lakh
large_withdrawals = transactions_df.filter(col(" WITHDRAWAL AMT ") > 100000).select("Account No", " WITHDRAWAL AMT ")
print("Customers with large withdrawals:")
large_withdrawals.show()

Customers with large withdrawals:
+-------------+----------------+
|   Account No| WITHDRAWAL AMT |
+-------------+----------------+
|409000611074'|        133900.0|
|409000611074'|        195800.0|
|409000611074'|        143800.0|
|409000611074'|        331650.0|
|409000611074'|        129000.0|
|409000611074'|        230013.0|
|409000611074'|        367900.0|
|409000611074'|        108000.0|
|409000611074'|        141000.0|
|409000611074'|        206000.0|
|409000611074'|        242300.0|
|409000611074'|        113250.0|
|409000611074'|        206900.0|
|409000611074'|        276000.0|
|409000611074'|        171000.0|
|409000611074'|        189800.0|
|409000611074'|        271323.0|
|409000611074'|        200600.0|
|409000611074'|        176900.0|
|409000611074'|        150050.0|
+-------------+----------------+
only showing top 20 rows



In [0]:
# Stop SparkSession
spark.stop()